In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# **Imports**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [3]:
sns.set_style('whitegrid')
sns.set(rc={'figure.figsize':(15, 5)})

# **Cargamos el dataset**

In [4]:
filename = "dataset_inf_telec_v1.csv"

In [5]:
df = pd.read_csv(
    filename,
    dtype={
        'ID_EQUIPO': int,
        'PUNTO_MEDICION': str,
        'CAPACIDAD_MAXIMA_GBS': float,
        'PASO': int,
        'LATENCIA_MS': float,
        'PORCENTAJE_PACK_LOSS': float,
        'INBOUND_BITS': np.float64,
        'OUTBOUND_BITS': np.float64,
        'MEDIDA': str,
    },
    parse_dates=[
        'FECHA_INICIO_MEDICION',
        'FECHA_FIN_MEDICION',
        'FECHA_HORA',
    ],
    na_values=['NaN']
)

## **Exploremos un poco el Dataset**

In [6]:
# Veamos el tamaño del dataset
print(f"Shape = {df.shape}")

Shape = (19680, 12)


Tenemos en total:
* +19K registros
* 12 columnas (features)

In [7]:
# Veamos una muestra de 10 registros del dataset
df.sample(10)

,ID_EQUIPO,PUNTO_MEDICION,CAPACIDAD_MAXIMA_GBS,FECHA_INICIO_MEDICION,FECHA_HORA,FECHA_FIN_MEDICION,PASO,LATENCIA_MS,PORCENTAJE_PACK_LOSS,INBOUND_BITS,OUTBOUND_BITS,MEDIDA
15393,4,NOC - Pilar,1.0,2020-06-04 17:00:00,2020-06-06 07:00:00,2020-06-21 19:00:00,7200,2.96155,0.00000,7.456840e+05,1.899935e+06,MB
12291,24,EDC - RDB,20.0,2020-06-04 17:00:00,2020-06-21 03:00:00,2020-06-21 19:00:00,7200,1.71228,0.00000,1.312720e+09,1.104970e+10,GB
12845,28,JM - Totoral Nueva,1.0,2020-06-04 17:00:00,2020-06-16 01:00:00,2020-06-21 19:00:00,7200,1.81214,0.00000,2.182929e+03,2.683384e+04,MB
13546,62,NOC - 6720HI to ETC,10.0,2020-06-04 17:00:00,2020-06-06 03:00:00,2020-06-21 19:00:00,7200,2.37634,0.00000,7.078218e+06,1.057888e+08,GB
16707,1,RDB - Escuela de Capacitacion,1.0,2020-06-04 17:00:00,2020-06-13 07:00:00,2020-06-21 19:00:00,7200,1.21703,0.38576,7.926736e+05,1.027501e+06,MB
16001,4,NOC - SW Clientes 2,1.0,2020-06-04 17:00:00,2020-06-05 17:00:00,2020-06-21 19:00:00,7200,1.47918,0.00000,1.145102e+07,8.845153e+07,MB
4131,62,NOC - 6720HI to R4 Silica,1.0,2020-06-04 17:00:00,2020-06-07 09:00:00,2020-06-21 19:00:00,7200,0.58038,0.00000,7.611542e+06,1.064550e+08,MB
16228,4,NOC - Switch Servers,1.0,2020-06-04 17:00:00,2020-06-07 13:00:00,2020-06-21 19:00:00,7200,1.97368,0.00000,8.706618e+04,3.403273e+07,MB
15960,4,NOC - SW Clientes 1,1.0,2020-06-04 17:00:00,2020-06-19 09:00:00,2020-06-21 19:00:00,7200,1.49348,0.00000,1.291335e+07,2.183006e+07,MB
2399,24,EDC - RDB,20.0,2020-06-04 17:00:00,2020-06-16 19:00:00,2020-06-21 19:00:00,7200,1.75342,0.11668,1.558304e+09,1.226332e+10,GB


In [8]:
# Veamos los tipos de las columnas
df.dtypes

ID_EQUIPO                         int64
PUNTO_MEDICION                   object
CAPACIDAD_MAXIMA_GBS            float64
FECHA_INICIO_MEDICION    datetime64[ns]
FECHA_HORA               datetime64[ns]
FECHA_FIN_MEDICION       datetime64[ns]
PASO                              int64
LATENCIA_MS                     float64
PORCENTAJE_PACK_LOSS            float64
INBOUND_BITS                    float64
OUTBOUND_BITS                   float64
MEDIDA                           object
dtype: object

#### Las columnas representan:
* **ID_EQUIPO**:             Switch/Router
* **PUNTO_MEDICION**:        Línea Troncal
* **CAPACIDAD_MAXIMA_GBS**:  Capacidad maxima [Gigabit/seg]
* **FECHA_INICIO_MEDICION**: Fecha y hora inicio medicion
* **FECHA_HORA**:            Fecha y hora efectiva de la medida
* **FECHA_FIN_MEDICION**:    Fecha y hora fin de medicion
* **PASO**:
* **LATENCIA_MS**:           Latencia (tiempo de espera de la respuesta) [mseg]
* **PORCENTAJE_PACK_LOSS**:  % de paquetes perdidos (pings configurados en diferentes momentos del dia) [0 a 100]
* **INBOUND_BITS**:          Trafico de entrada [Bit/seg]
* **OUTBOUND_BITS**:         Trafico de salida  [Bit/seg]
* **MEDIDA**:                Unidad a convertir los campos `INBOUND_BITS` y `OUTBOUND_BITS`

In [9]:
# Veamos una descripcion general de las variables numericas del Dataset
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID_EQUIPO,19680.0,2.525000e+01,1.742947e+01,1.000000,1.100000e+01,2.400000e+01,3.100000e+01,6.200000e+01
CAPACIDAD_MAXIMA_GBS,19680.0,6.211654e+00,8.264031e+00,0.027263,1.000000e+00,1.000000e+00,1.000000e+01,4.000000e+01
PASO,19680.0,7.200000e+03,0.000000e+00,7200.000000,7.200000e+03,7.200000e+03,7.200000e+03,7.200000e+03
LATENCIA_MS,19680.0,2.821440e+00,2.134160e+00,0.250300,1.281635e+00,2.034790e+00,3.566360e+00,2.705176e+01
PORCENTAJE_PACK_LOSS,19680.0,2.034690e-01,9.123203e-01,0.000000,0.000000e+00,0.000000e+00,1.474400e-01,4.152227e+01
INBOUND_BITS,19680.0,6.530502e+08,2.381826e+09,101.856197,1.294885e+06,1.489632e+07,2.271407e+08,2.418785e+10
OUTBOUND_BITS,19680.0,1.335330e+09,3.099656e+09,3633.833278,3.018778e+06,8.310589e+07,9.893677e+08,2.327810e+10


In [10]:
# Veamos una descripcion general de las variables categoricas del Dataset
df.describe(include='O').T

,count,unique,top,freq
PUNTO_MEDICION,19680,48,RDB - Escuela de Capacitacion,410
MEDIDA,19680,2,MB,11890


## **Importante**

#### Cada `ID_EQUIPO` puede tener asociado uno o más `PUNTO_MEDICION`

In [11]:
# Veamos la cantidad de registros por ID_EQUIPO.
df.ID_EQUIPO.value_counts()

4     3690
24    3280
31    2460
62    2460
11    2050
23    1640
30    1230
28     820
41     820
25     820
1      410
Name: ID_EQUIPO, dtype: int64

In [12]:
# Veamos la cantidad de registros por PUNTO_MEDICION.
df.PUNTO_MEDICION.value_counts()

RDB - Escuela de Capacitacion     410
Carlos Paz - Cosquin              410
NOC - Switch Servers              410
NOC - 6720HI to ETC               410
Carlos Paz - La Falda             410
NOC - Pilar                       410
EDC - MOP                         410
NOC - 6720HI to N20-1             410
Yocsina - Mogote                  410
NOC - S9306 to SS6720HI           410
NOC - ACHALA - Solo Dolores       410
NOC - Interfabricas               410
NOC - 6720HI to EDC               410
RDB - PEA                         410
NOC - ET Sur                      410
EDC - Telecomunicacioes           410
NOC - 6720HI to R4 Silica         410
EDC - ET Oeste                    410
SF - SF Adm                       410
JM - Totoral Nueva                410
ABA - Temple                      410
BAZ - Yocsina                     410
RDB - GZU                         410
SF - Las Varillas                 410
JM - Totoral Vieja                410
EDC - Capitalinas                 410
NOC - SW Cli

In [13]:
for punto_medicion in list(df.PUNTO_MEDICION.unique()):
    print(punto_medicion)
    start_measurings = df[df.PUNTO_MEDICION == punto_medicion].FECHA_INICIO_MEDICION.unique()
    end_measurings = df[df.PUNTO_MEDICION == punto_medicion].FECHA_FIN_MEDICION.unique()
    print(f"\t Inicio de Medicion        : {start_measurings}")
    print(f"\t Fin de Medicion           : {end_measurings}")

    tdelta = end_measurings[0] - start_measurings[0]
    days = tdelta.astype('timedelta64[D]')
    print(f"\t Duracion de la Medicion   : {days / np.timedelta64(1, 'D')} dias")
    print("=" * 20)

ABA - Abasto Cliente
	 Inicio de Medicion        : ['2020-06-04T17:00:00.000000000']
	 Fin de Medicion           : ['2020-06-21T19:00:00.000000000']
	 Duracion de la Medicion   : 17.0 dias
ABA - Temple
	 Inicio de Medicion        : ['2020-06-04T17:00:00.000000000']
	 Fin de Medicion           : ['2020-06-21T19:00:00.000000000']
	 Duracion de la Medicion   : 17.0 dias
BAZ - Carlos Paz
	 Inicio de Medicion        : ['2020-06-04T17:00:00.000000000']
	 Fin de Medicion           : ['2020-06-21T19:00:00.000000000']
	 Duracion de la Medicion   : 17.0 dias
BAZ - Yocsina
	 Inicio de Medicion        : ['2020-06-04T17:00:00.000000000']
	 Fin de Medicion           : ['2020-06-21T19:00:00.000000000']
	 Duracion de la Medicion   : 17.0 dias
Carlos Paz - Cosquin
	 Inicio de Medicion        : ['2020-06-04T17:00:00.000000000']
	 Fin de Medicion           : ['2020-06-21T19:00:00.000000000']
	 Duracion de la Medicion   : 17.0 dias
Carlos Paz - La Falda
	 Inicio de Medicion        : ['2020-06-04T17:00:00.

### Notar que la duracion de la medicion por cada `PUNTO_MEDICION` es de 17 dias